# Importing required librarie Pipeline#


In [1]:
print("Step 1: Required librarie imported successfully")


import time
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

from imblearn.over_sampling import SMOTE

from imblearn.pipeline import Pipeline as IMBPipeline
from sklearn.pipeline import Pipeline

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB



Step 1: Required librarie imported successfully


#To ignore warning

In [2]:
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)



# Loading online_shoppers_intention.csv dataset#

In [3]:
print("Step 2: Created DataFrame successfully")

df = pd.read_csv("online_shoppers_intention.csv")



Step 2: Created DataFrame successfully


# Feature Engineering#


In [4]:
print("Step 3: Feature Engineering Done successfully on Weekend, Revenue")


df['Weekend'] = df['Weekend'].replace((True, False), (1, 0))
df['Revenue'] = df['Revenue'].replace((True, False), (1, 0))

condition = df['VisitorType'] == 'Returning_Visitor'



Step 3: Feature Engineering Done successfully on Weekend, Revenue


# Added Returning_Visitor column#

In [5]:
print("Step 4: Added Returning_Visitor column successfully")

df['Returning_Visitor'] = np.where(condition, 1, 0)

df = df.drop(columns=['VisitorType'])


Step 4: Added Returning_Visitor column successfully


# Applying One Hot Encoding on Month column#

In [6]:
print("Step 5: Applied one hot encoding successfully on Month column")


ordinal_encoder = OrdinalEncoder()
df['Month'] = ordinal_encoder.fit_transform(df[['Month']])



Step 5: Applied one hot encoding successfully on Month column


# Checking correlation on Revenue column#

In [7]:
print("Step 6: Checking correlation done successfully")


result = df[df.columns[1:]].corr()['Revenue']						
result1 = result.sort_values(ascending=False)


Step 6: Checking correlation done successfully


# Prepairing Features as X and target as y#

In [8]:
print("Step 7: Prepairing features as X and target as y done successfully")


X = df.drop(['Revenue'], axis=1)
y = df['Revenue']


Step 7: Prepairing features as X and target as y done successfully


# Prepairing Train and Test Dataset#

In [9]:
print("Step 8: Splitting data X_train, X_test, y_train & y_test done successfully")


X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size = 0.3, 
    random_state = 0
)



Step 8: Splitting data X_train, X_test, y_train & y_test done successfully


# Model Pipeline#

In [10]:
print("Step 9: model_pipeline fcuntion created done successfully")

def model_pipeline(X, model):
    n_c = X.select_dtypes(exclude=['object']).columns.tolist()
    c_c = X.select_dtypes(include=['object']).columns.tolist()

    numeric_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='constant')),
        ('scaler', MinMaxScaler())
    ])

    categorical_pipeline = Pipeline([
        ('encoder', OneHotEncoder(handle_unknown='ignore'))
    ])
    
    preprocessor = ColumnTransformer([
        ('numeric', numeric_pipeline, n_c),
        ('categorical', categorical_pipeline, c_c)
    ], remainder='passthrough')

    final_steps = [
        ('preprocessor', preprocessor),
        ('smote', SMOTE(random_state=1)),
        ('feature_selection', SelectKBest(score_func = chi2, k = 6)),
        ('model', model)
    ]

    return IMBPipeline(steps = final_steps)  # Ensure to use IMBPipeline if using imblearn's Pipeline



Step 9: model_pipeline fcuntion created done successfully


# Model Selection#

In [11]:
print("Step 10: select_model fcuntion created done successfully")


def select_model(X, y, pipeline=None):

    classifiers = {}
    

    c_d1 = {"RandomForestClassifier": RandomForestClassifier()}
    classifiers.update(c_d1)

    c_d2 = {"DecisionTreeClassifier": DecisionTreeClassifier()}
    classifiers.update(c_d2)

    c_d3 = {"KNeighborsClassifier": KNeighborsClassifier()}
    classifiers.update(c_d3)

    c_d4 = {"RidgeClassifier": RidgeClassifier()}
    classifiers.update(c_d4)

    c_d5 = {"BernoulliNB": BernoulliNB()}
    classifiers.update(c_d5)

    c_d6 = {"SVC": SVC()}
    classifiers.update(c_d6)
    
    
   
    cols = ['model', 'run_time', 'roc_auc']
    df_models = pd.DataFrame(columns = cols)

    for key in classifiers:
        
        start_time = time.time()
        
        print()
        print("Step 12: model_pipeline run successfully on", key)

        pipeline = model_pipeline(X, classifiers[key])
        
        cv = cross_val_score(pipeline, X, y, cv=10, scoring='roc_auc')

        row = {'model': key,
               'run_time': format(round((time.time() - start_time)/60,2)),
               'roc_auc': cv.mean(),
        }

        df_models = pd.concat([df_models, pd.DataFrame([row])], ignore_index=True)
        
    df_models = df_models.sort_values(by='roc_auc', ascending=False)
	
    return df_models


Step 10: select_model fcuntion created done successfully


# Access Model select_model function#


In [12]:
print("Step 11: Accessing select_model function done successfully")


models = select_model(X_train, y_train)



Step 11: Accessing select_model function done successfully

Step 12: model_pipeline run successfully on RandomForestClassifier

Step 12: model_pipeline run successfully on DecisionTreeClassifier

Step 12: model_pipeline run successfully on KNeighborsClassifier

Step 12: model_pipeline run successfully on RidgeClassifier

Step 12: model_pipeline run successfully on BernoulliNB

Step 12: model_pipeline run successfully on SVC


# Lets see total model with score #

In [13]:
print("Step 13: Accessing select_model function done successfully")
print()

print(models)

Step 13: Accessing select_model function done successfully

                    model run_time   roc_auc
5                     SVC     1.07  0.889927
0  RandomForestClassifier     0.64  0.886405
4             BernoulliNB     0.03  0.870663
3         RidgeClassifier     0.02  0.856459
2    KNeighborsClassifier     0.03  0.840885
1  DecisionTreeClassifier     0.05  0.734329


# Accessing best model and training #

In [14]:
print()
print("Step 14: Accessing select_model function done successfully")

selected_model = SVC()
bundled_pipeline = model_pipeline(X_train, selected_model)
bundled_pipeline.fit(X_train, y_train)




Step 14: Accessing select_model function done successfully


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('numeric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='constant')),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['Administrative',
                                                   'Administrative_Duration',
                                                   'Informational',
                                                   'Informational_Duration',
                                                   'ProductRelated',
                                                   'ProductRelated_Duration',
                                                   'BounceRates', 'ExitRates',
                                                   'PageValues', 'SpecialDay',
                                                   'Month', 'OperatingSystems',
                                                   'Browser', 'Region',
                                                   'TrafficType', 'Weekend',
                                                   'Returning_Visitor']),
                                                 ('categorical',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  [])])),
                ('smote', SMOTE(random_state=1)),
                ('feature_selection',
                 SelectKBest(k=6,
                             score_func=<function chi2 at 0x0000021C70E78040>)),
                ('model', SVC())])

# Accessing best model and training #

In [15]:
print("Step 15: Results predicted successfully")

y_pred = bundled_pipeline.predict(X_test)



Step 15: Results predicted successfully


# ROC and AOC score #

In [16]:

print("Step 16: ROC and AOC scores")

roc_auc = roc_auc_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
f1_score = f1_score(y_test, y_pred)

print()
print('ROC/AUC:', roc_auc)
print('Accuracy:', accuracy)
print('F1 score:', f1_score)
print()





Step 16: ROC and AOC scores

ROC/AUC: 0.7872136596906621
Accuracy: 0.8775344687753447
F1 score: 0.6413301662707839

